In [118]:
#ML final project
#Yu Zhou, Che Wang

from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
import numpy as np
import keras
from keras import backend as K
import matplotlib.image as mpimg
from glob import glob
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# A bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline 
import matplotlib.pyplot as plt
from scipy import ndimage
from scipy import misc
"""datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')"""
#don't worry about data augmentation for now...
#data preprocessing is done in another file
print()

In [123]:

globPaths = ['data/c0small/*.jpg','data/c1small/*.jpg']
num_classes = 2
numTrain = 1000

X_train = np.empty((2000,64,64,3))
X_test = np.empty((600,64,64,3))
y_train =np.empty((2000))
y_test = np.empty((600))

i = 0
j = 0

for folderIndex in range(2):
    index = 0
    filelist = glob(globPaths[folderIndex])
    for filename in filelist:
        newImg = misc.imread(filename)
        if index<numTrain:
            X_train[i] = newImg
            y_train[i] = folderIndex
            i+=1
        else:
            X_test[j] = newImg
            y_test[j] = folderIndex
            j+=1
        index+=1
        
        
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
#then data normalization
X_train /= 255
X_test /= 255
        
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)
    
print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)
print("data read and normalization finished!")



(2000, 64, 64, 3) (600, 64, 64, 3)
(2000,) (600,)
data read and normalization finished!


In [127]:
# input image dimensions
img_rows, img_cols = 64, 64
img_channels = 3
#the later are about training...

input_shape = (img_rows, img_cols,img_channels)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),padding= 'same',
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes, activation='softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.001, decay=1e-6)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt, #keras.optimizers.Adadelta(0.01),
              metrics=['accuracy'])
print("model building and compiling finished!")


model building and compiling finished!


In [129]:
batch_size = 32 
nb_epoch = 5
model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=nb_epoch,
          verbose=1,
          validation_data=(X_test, Y_test))

pred = model.predict(X_train,batch_size=10)
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save('keras_cnn_model.h5')

Train on 2000 samples, validate on 600 samples
Epoch 1/5
2000/2000 [==============================] - 67s - loss: 0.1128 - acc: 0.9615 - val_loss: 0.0322 - val_acc: 0.9917
Epoch 2/5
2000/2000 [==============================] - 64s - loss: 0.1370 - acc: 0.9615 - val_loss: 0.1479 - val_acc: 0.9483
Epoch 3/5
2000/2000 [==============================] - 66s - loss: 0.1113 - acc: 0.9650 - val_loss: 0.0933 - val_acc: 0.9617
Epoch 4/5
2000/2000 [==============================] - 65s - loss: 0.0659 - acc: 0.9795 - val_loss: 0.3999 - val_acc: 0.8633
Epoch 5/5
2000/2000 [==============================] - 71s - loss: 0.0678 - acc: 0.9785 - val_loss: 0.0087 - val_acc: 0.9967
Test loss: 0.00868155636883
Test accuracy: 0.996666666667
